# Тест загрузки по месяцам и кварталам

Этот notebook демонстрирует загрузку данных по месяцам и кварталам.

In [ ]:
# ВАЖНО: Автоперезагрузка модулей при изменениях
%load_ext autoreload
%autoreload 2

In [ ]:
# Настройка путей
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"Корень проекта: {project_root}")

In [ ]:
# Импорты
from src.data.load_data import load_raw_data
import polars as pl
import inspect

print("✓ Импорты успешны")

In [ ]:
# Проверка сигнатуры функции
print("Сигнатура load_raw_data:")
print(inspect.signature(load_raw_data))
print("\nДолжно быть: (year: int, month: int = None, quarter: int = None)")

## 1. Загрузка всего года

In [ ]:
# Загружаем весь 2024 год (все 12 месяцев)
print("Загрузка всего 2024 года...")
trips_year = load_raw_data(year=2024)

print(f"\n✓ Загружено: {trips_year.shape[0]:,} поездок")
print(f"✓ Колонок: {trips_year.shape[1]}")
print(f"\nПервые строки:")
trips_year.head()

## 2. Загрузка конкретного месяца

In [ ]:
# Загружаем только январь 2024
print("Загрузка января 2024...")
trips_jan = load_raw_data(year=2024, month=1)

print(f"\n✓ Январь 2024: {trips_jan.shape[0]:,} поездок")
trips_jan.head()

In [ ]:
# Загружаем июль 2024
print("Загрузка июля 2024...")
trips_jul = load_raw_data(year=2024, month=7)

print(f"\n✓ Июль 2024: {trips_jul.shape[0]:,} поездок")

## 3. Загрузка квартала

In [ ]:
# Загружаем Q2 2018 (квартальный файл)
print("Загрузка Q2 2018 (квартальный файл)...")
trips_q2_2018 = load_raw_data(year=2018, quarter=2)

print(f"\n✓ Q2 2018: {trips_q2_2018.shape[0]:,} поездок")
trips_q2_2018.head()

In [ ]:
# Загружаем Q1 2024 (через месяцы: янв+фев+март)
print("Загрузка Q1 2024 (через месяцы)...")
trips_q1_2024 = load_raw_data(year=2024, quarter=1)

print(f"\n✓ Q1 2024: {trips_q1_2024.shape[0]:,} поездок")
print("  (Это сумма января, февраля и марта)")

## 4. Сравнение: проверка что квартал = 3 месяца

In [ ]:
# Загружаем Q1 через функцию quarter
q1_via_quarter = load_raw_data(year=2024, quarter=1)

# Загружаем те же месяцы по отдельности
jan = load_raw_data(year=2024, month=1)
feb = load_raw_data(year=2024, month=2)
mar = load_raw_data(year=2024, month=3)

# Объединяем
q1_via_months = pl.concat([jan, feb, mar], how='diagonal')

print("Сравнение загрузки Q1 2024:")
print(f"  Через quarter=1: {q1_via_quarter.shape[0]:,} поездок")
print(f"  Через месяцы:    {q1_via_months.shape[0]:,} поездок")
print(f"  Совпадает: {q1_via_quarter.shape[0] == q1_via_months.shape[0]}")

## 5. Статистика по всем кварталам 2024

In [ ]:
# Загружаем все кварталы
quarters_2024 = {}

for q in [1, 2, 3, 4]:
    df = load_raw_data(year=2024, quarter=q)
    quarters_2024[f'Q{q}'] = df.shape[0]

print("Поездок по кварталам 2024:")
for quarter, count in quarters_2024.items():
    print(f"  {quarter}: {count:,} поездок")

total = sum(quarters_2024.values())
print(f"\nВсего за год: {total:,} поездок")

## 6. Статистика по месяцам 2024

In [ ]:
# Загружаем все месяцы
months_2024 = {}
month_names = ['Янв', 'Фев', 'Мар', 'Апр', 'Май', 'Июн', 
               'Июл', 'Авг', 'Сен', 'Окт', 'Ноя', 'Дек']

for month in range(1, 13):
    df = load_raw_data(year=2024, month=month)
    months_2024[month_names[month-1]] = df.shape[0]

print("Поездок по месяцам 2024:")
for month, count in months_2024.items():
    print(f"  {month}: {count:,} поездок")

total = sum(months_2024.values())
print(f"\nВсего за год: {total:,} поездок")

## 7. Визуализация (опционально)

In [ ]:
# Если есть matplotlib
try:
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(12, 5))
    
    # График по месяцам
    plt.bar(months_2024.keys(), months_2024.values())
    plt.title('Количество поездок по месяцам 2024')
    plt.xlabel('Месяц')
    plt.ylabel('Количество поездок')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("matplotlib не установлен, пропускаем визуализацию")

## ✅ Итоги

Мы успешно протестировали:
- ✅ Загрузку всего года
- ✅ Загрузку конкретного месяца
- ✅ Загрузку квартала (старый и новый форматы)
- ✅ Проверили что квартал = сумма 3 месяцев
- ✅ Получили статистику по всем периодам

Все работает корректно! 🎉